In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
secret_key = 'masked'


In [3]:
df = pd.read_csv('Crops_AllData_Normalized.csv',encoding='latin-1')
print(df.shape)
df.head()

(692, 11)


,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1975,1975,ha,0.0,F
1,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1976,1976,ha,5900.0,F
2,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1977,1977,ha,6000.0,F
3,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1978,1978,ha,6000.0,F
4,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1979,1979,ha,6000.0,F


In [4]:
df.rename(columns={'Area Code':'area_code',
                   'Area':'area',
                   'Item Code':'item_code',
                   'Item':'item',
                   'Element Code':'element_code',
                   'Element':'element',
                   'Year Code':'year_code',
                   'Year':'year',
                   'Unit':'unit',
                   'Value':'value',
                   'Flag':'flag'
},inplace=True)

In [5]:
conn = sqlite3.connect('agri_db.sqlite')
c = conn.cursor()

In [6]:
c.execute('CREATE TABLE IF NOT EXISTS crop_data (area_code int,area text,item_code int, item text,element_code int,element text,year_code int, year int,unit text,value float,flag text)')
conn.commit()

In [7]:
df.to_sql('agri_data',conn,if_exists='replace',index=False)
c.execute('''
SELECT item FROM agri_data limit 100 
          ''')
for i in c.fetchall():
    print(i)

('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)
('Almonds, with shell',)


In [8]:
def read_sql_query(sql,db):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row)
    conn.close()
    

In [9]:
read_sql_query('SELECT * FROM agri_data where value > 6000.0 limit 10','agri_db.sqlite')

(2, 'Afghanistan', 221, 'Almonds, with shell', 5312, 'Area harvested', 1990, 1990, 'ha', 6037.0, 'Im')
(2, 'Afghanistan', 221, 'Almonds, with shell', 5312, 'Area harvested', 2000, 2000, 'ha', 7000.0, 'F')
(2, 'Afghanistan', 221, 'Almonds, with shell', 5312, 'Area harvested', 2001, 2001, 'ha', 9000.0, 'F')
(2, 'Afghanistan', 221, 'Almonds, with shell', 5312, 'Area harvested', 2004, 2004, 'ha', 12000.0, '*')
(2, 'Afghanistan', 221, 'Almonds, with shell', 5312, 'Area harvested', 2005, 2005, 'ha', 11768.0, 'Im')
(2, 'Afghanistan', 221, 'Almonds, with shell', 5312, 'Area harvested', 2006, 2006, 'ha', 12000.0, None)
(2, 'Afghanistan', 221, 'Almonds, with shell', 5312, 'Area harvested', 2007, 2007, 'ha', 12000.0, None)
(2, 'Afghanistan', 221, 'Almonds, with shell', 5312, 'Area harvested', 2008, 2008, 'ha', 12000.0, None)
(2, 'Afghanistan', 221, 'Almonds, with shell', 5312, 'Area harvested', 2009, 2009, 'ha', 11029.0, None)
(2, 'Afghanistan', 221, 'Almonds, with shell', 5312, 'Area harvested',

In [10]:
from langchain.llms import OpenAI
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [11]:
input_db = SQLDatabase.from_uri('sqlite:///agri_db.sqlite')
llm_1 = OpenAI(temperature=0,openai_api_key=secret_key)
db_agent = SQLDatabaseChain(llm=llm_1,
                            database = input_db,
                            verbose=True)

C:\Users\anike\AppData\Local\Temp\ipykernel_8904\352095845.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm_1 = OpenAI(temperature=0,openai_api_key=secret_key)
c:\Users\anike\anaconda3\Lib\site-packages\langchain_experimental\sql\base.py:77: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [12]:
db_agent.run('What are the unique values in item column ?')

C:\Users\anike\AppData\Local\Temp\ipykernel_8904\4106515888.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  db_agent.run('What are the unique values in item column ?')




> Entering new SQLDatabaseChain chain...
What are the unique values in item column ?
SQLQuery:

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}